***CONECTANDO AO MONGO DB***

In [1]:
from pymongo.mongo_client import MongoClient

In [2]:
client = MongoClient()

try:
    client.admin.command('ping')
    print('conectou!!')
except Exception as e:
    print(e)

conectou!!


***CRIANDO BANCO E COLEÇÃO***

In [7]:
db = client["db_produtos_amazon"]

In [8]:
collection = db["produtos_amazon"]

In [9]:
teste = {
    "teste" : 1
}

collection.insert_one(teste)

InsertOneResult(ObjectId('67f47072a5ec024b96f198f8'), acknowledged=True)

In [11]:
client.list_database_names()

['admin',
 'config',
 'db_avaliacao_credito',
 'db_cotacoes',
 'db_dados_eleitorais',
 'db_eleicao_pe',
 'db_pipeline_abandono_escolar',
 'db_produtos',
 'db_produtos_amazon',
 'db_reclamacoes',
 'local']

***COMEÇANDO A USAR O SPARK***

In [12]:
import os
from pyspark.sql import SparkSession

In [13]:
os.environ["SPARK_HOME"] = "/opt/spark/spark-3.5.5-bin-hadoop3"

In [14]:
import findspark

In [15]:
findspark.init()

In [16]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

25/04/07 21:44:17 WARN Utils: Your hostname, dg-IdeaPad-3-15ALC6 resolves to a loopback address: 127.0.1.1; using 192.168.0.19 instead (on interface wlp1s0)
25/04/07 21:44:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/07 21:44:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
spark

***lENDO OS DADOS***

In [1]:
path_csv = "../data_raw/amz_br_total_products_data_processed.csv"

***TRANSFORMANDO PARA JSON***